#### Regresión lineal 
El dataset en cuestión es 'advertising_and_sales' que contiene los ingresos obtenidos de ventas junto con tres variables númericas que indican presupuestos invertidos en diferentes medios de promoción y una variable categórica que describe el nivel de popularidad del o los influencers que participaron en la publicidad. Este ejemplo se basará en desarrollar un modelo lineal que se ajuste a una sola variable independiente con la variable objetivo.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from scipy.stats import pearsonr, zscore, kurtosis, norm, shapiro
from sklearn.linear_model import LinearRegression


df = pd.read_csv("data/advertising_and_sales.csv")
df.set_index("id", inplace=True)

df

#### Análisis Descriptivo
Si bien definos centrarnos solamente en variables continuas, a continuación se hará una visuliazación y análisis multivariable, relacionando valores y categorías de manera que se obtengan los siguientes resultados:
- Los números en ventas varían y se concentran aproximadamente entre 100 y 300 millones
- Hay un incremento de los ingresos conforme aumenta el presupuesto de promoción en TV y Radio
- Los distintos niveles de popularidad en Influencers no suelen influir en las ingresos

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,5))

sns.scatterplot(df, x="tv", y="sales", hue="influencer", ax=ax[0])
sns.scatterplot(df, x="radio", y="sales", hue="influencer", ax=ax[1])
sns.scatterplot(df, x="social_media", y="sales", hue="influencer", ax=ax[2])
ax[0].grid("on")
ax[1].grid("on")
ax[2].grid("on")

plt.show()

#### Análisis Exploratorio de datos

Son un conjunto de métodos que utilizan conceptos estadísticos enfocados en explorar y analizar el comportamiento de los datos donde el objetivo es descubrir patrones, relaciones y estructuras que guíen la implementación de un modelo particular como solución. Algunos puntos clave en un caso de regresión son:

Distribución Normal: su presencia en las variables continuas es fundamentalmente requerida para algunos modelos regresores debido a la estructura que genera a la hora de ajustarse a los datos, existen diferentes formas de estimar una distribución de este tipo, como medias y medianas que coinciden, representaciones gráficas con forma simétrica respecto a la media y acampanada con la mayoría de los valores concetrados, sin colas pesadas o nulas(curtosis mesocúritca, cercana a 0).

Correlación Lineal: valor estadístico que indica la fuerza y dirección de una asociación lineal entre dos variables númericas, es una de las medidas estadísticas más importantes y una correlación lineal puede llegar a explicar la influecia de una variable independiente sobre una dependiente.

Outliers: valores que destacan por su notoria distinción(mayor o menor) a los demás del conjunto. La forma más efectiva de hallar estos últimos es la Regla Empírica, según la cual los datos que se encuentran dentro de 3 desviaciones éstandar con respecto a la media representan el 99.7% de los datos, por lo que podemos concluir con bastante seguridad que los datos cuyos valores Z caen por afuera de este umbral son atípicos, ya que son distintos al 99.7%.


In [ ]:
corr, _ = pearsonr(df["radio"], df["sales"])

df_zscore = df[["tv","sales"]]

df_zscore["tv_zscore"] = zscore(df["tv"]).abs()
df_zscore["sales_zscore"] = zscore(df["sales"]).abs()

outliers = df_zscore.loc[(df_zscore["tv_zscore"] > 3) | (df_zscore["sales_zscore"] > 3),["tv","sales"]]

curtosis_x = kurtosis(df["tv"])
curtosis_y = kurtosis(df["sales"])

print("----------------------------------------------------------")
print(f"Correlación Pearsonr: {round(corr,2)}")
print(f"Outliers: {outliers.shape[0]}")
print(f"Curtosis variable X: {round(curtosis_x,1)} | Curtosis variable Y: {round(curtosis_y,1)}")
print("----------------------------------------------------------")

#### Pruebas de normalidad
Los métodos estadísticos mencionados anteriormente son ampliamente utilizados y dan una base sólida para diagnosticar, sin embargo, las siguientes pruebas de hipótesis proporcionan una evaluación más precisa y son menos propensas a errores de confirmación

Shapiro-Wilk: Se basa en la correlación entre los valores del conjunto y una versión de estos que representan un respectivo de nivel de desviación éstandar, que se denominan valores Z o estandarizados.

Kolmogorov-Smirnov: Compara la función de distribución acumulativa(CDF) del conjunto de valores con la de una distribución normal.

Ambas pruebas hacen uso del valor P para concluir su hipótesis nula(los datos siguen una distribución normal asimilada) o su hipótesis alternativa(caso contrario) con un valor P muy cercano, igual o menor a 0.05.

In [ ]:
df["tv_zscore"] = zscore(df["tv"])
df["sales_zscore"] = zscore(df["sales"])

cdf_Trip_Distance_km = norm.cdf(df["tv"], df["tv"].mean(), df["tv"].std()) 
cdf_Trip_Price = norm.cdf(df["sales"], df["sales"].mean(), df["sales"].std()) 

fig = make_subplots(rows=2, cols=2)

fig.add_trace(go.Scatter(x=df["tv_zscore"], y=df["tv"], mode="markers", name="Presupuesto en TV"), row=1,col=1)
fig.add_trace(go.Scatter(x=df["sales_zscore"], y=df["sales"], mode="markers", name="Ingresos de ventas"), row=1,col=2)
fig.add_trace(go.Scatter(x=df["tv"].values, y=cdf_Trip_Distance_km, mode="markers", name="Presupuesto en TV"), row=2,col=1)
fig.add_trace(go.Scatter(x=df["sales"].values, y=cdf_Trip_Price, mode="markers", name="Ingresos de ventas"), row=2,col=2)
fig.update_layout(height=600, width=850, title_text="Shapiro-wilk & Kolmogorov-smirnov")

_, p_value_var_x = shapiro(df["tv"])
_, p_value_var_y = shapiro(df["sales"])

fig

#### Modelo lineal

Luego de haber analizado la relación lineal y distribuciones de ambas variables continuas y obtener resultados como una correlación lineal igual 0.87, distribuciones platicúrticas(más planas que una gaussiana) y nulos valores atípicos se procede a generar el modelo lineal y ponerlo en práctica.

In [110]:
linear_regression = LinearRegression()
 
var_x = df["tv"].values.reshape((-1,1)) # variable X en la correlación lineal
var_y = df["sales"] # variable Y en la correlación lineal

linear_regression.fit(var_x, var_y)

# regresión de 10 objetos aleatorios

objects = df["tv"].sample(n=10).to_numpy().reshape((-1,1))

predicts = linear_regression.predict(objects)

#### Dashboard que refleja los insigths obtenidos y el ajuste del modelo lineal sobre los datos

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e2_body",children=[
    html.H1("Marketing sobre Ventas",id="title",className="e2_title"),
    html.Div(id="dashboard",className="e2_dashboard",children=[
        html.Div(id="column-1",className="e2_column_1",children=[
            dcc.Dropdown(id="dropdown",className="e2_dropdown",
                        options=[
                            {"label":"Presupuesto en TV","value":"tv"},
                            {"label":"Ingresos de ventas","value":"sales"}
                        ],
                        value="tv",
                        multi=False,
                        clearable=False),
            html.Div(className="e2_div_graphs",children=[
                dcc.Graph(id="graph-1",className="e2_graphs",figure={}), 
                dcc.Graph(id="graph-2",className="e2_graphs",figure={})
            ])
        ]),
        html.Div(id="column-2",className="e2_column_2",children=[
            html.H2("Shapiro-Wilk Valor-P",id="p_values_title",className="e2_p_values_title"),
            html.Div(id="p_values",className="e2_stats_div",children=[
                html.Div(id="p_value_var_x",className="e2_stats",children=[html.P(f"Variable X: {round(p_value_var_x,2)}",style={"font-size":"1em"})]),
                html.Div(id="p_value_var_y",className="e2_stats",children=[html.P(f"Variable Y: {round(p_value_var_y,2)}",style={"font-size":"0.98em"})])
            ]),
            html.Div(f"Correlación lineal: {round(corr,2)}",className="e2_corr",id="corr"),
            dcc.Graph(id="graph-3",className="e2_graph_3",figure={})
        ])
    ])
])

@app.callback(
    [Output(component_id="graph-1",component_property="figure"),
    Output(component_id="graph-2",component_property="figure"),
    Output(component_id="graph-3",component_property="figure")],
    [Input(component_id="dropdown",component_property="value")]
)

def update_dash(slct_var):
    
    mean = df[slct_var].mean()
    median = df[slct_var].median()
    
    extr_list = [0]
    
    var_title = "Presupuesto en TV"
    
    if slct_var == "Sales":
        extr_list.append(60)
        var_title = "Ingresos de ventas"
    elif slct_var == "TV":
        extr_list.append(40)
        var_title = "Presupuesto en TV"
    
    histplot = go.Figure(go.Histogram(x=df[slct_var],name="Distribución"))
    histplot.add_trace(go.Scatter(x=[mean,mean],y=extr_list,mode="lines+markers",marker_color="red",name="Media"))
    histplot.add_trace(go.Scatter(x=[median,median],y=extr_list,mode="lines+markers",marker_color="green",name="Mediana"))
    histplot.update_layout(title="Histograma",xaxis_title=var_title)
    
    boxplot = px.box(df, y=slct_var, points="all", title="Gráfico de caja")
    
    scatter = go.Figure()
    scatter.add_trace(go.Scatter(x=df["tv"],y=df["sales"],mode="markers",marker_color="blue",name="Ingrsos de ventas reales"))
    scatter.add_trace(go.Scatter(x=objects.reshape(-1),y=predicts,mode="lines+markers",marker_color="red",name="Predicciones"))
    scatter.update_layout(title="Regresión Lineal",xaxis_title="Presupuesto en TV",yaxis_title="Ingresos de ventas")

    return histplot, boxplot, scatter

if __name__ == "__main__":
    app.run_server(debug=False)

#### Error de bías
También denominado sesgo, es la diferencia entre la valores asumidos por el modelo y los valores verdaderos que llega a afectar la dirección de los resultados. Sucede principalmente en algoritmos paramétricos que suponen sobre ciertas estructuras o relaciones en los datos para una mejor interpretación. Alto bías: requiere más suposiciones a la hora de estimar la función objetivo, ejemplos de algoritmos con alto bías: Regresión Lineal, Regresión Logística, Naive Bayes, algoritmos de series temporales.

Este ejemplo ilustra como se asume principalmente que el incremento o decremento de una variable influye a otra, se puede observar en el último gráfico del dashboard la dispersión de los datos y como algunos se alejan del patrón lineal, mostrando algunos resultados que obtuvieron un valor similar en cuanto a ganancias de ventas pero habiendo utilizado un menor presupuesto de marketing, este efecto que fue definido en el anterior párrafo tiene la característica de tener un origen predecible y en el problema visto puede deberse a una inversión más participativa y equitativa en la Radio lo que ayudó al éxito financiero de algunos casos. En conclusión, en un caso económico como este un valor correlativo de ese nivel deja en claro una recompensa cuando se realizan promociones en la TV y el hecho de que algunos datos representen ingresos con menores gastos a comparación de otros lleva a conclusiones beneficiosas y una recapacitación a la hora de repartir los gastos en publicidad.